In [1]:
# walking through code from here: https://github.com/nikhilbarhate99/PPO-PyTorch

# MIT License

# Copyright (c) 2018 Nikhil Barhate

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.


In [2]:
import gym
from PIL import Image
import torch
import torch.nn as nn
from torch.distributions import Categorical
import os

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
class Memory:
    def __init__(self):
        self.actions = []
        self.states = []
        self.logprobs = []
        self.rewards = []
        self.is_terminals = []
        
    def clear_memory(self):
        del self.actions[:]
        del self.states[:]
        del self.logprobs[:]
        del self.rewards[:]
        del self.is_terminals[:]

In [4]:
class ActorCritic(nn.Module):
    def __init__(self, state_dim, action_dim, n_latent_var):
        super(ActorCritic, self).__init__()
        
        # actor
        self.action_layer = nn.Sequential(
            nn.Linear(state_dim, n_latent_var),
            nn.Tanh(),
            nn.Linear(n_latent_var, n_latent_var),
            nn.Tanh(),
            nn.Linear(n_latent_var, action_dim),
            nn.Softmax(dim=-1)
        )
        
        # critic
        self.value_layer = nn.Sequential(
            nn.Linear(state_dim, n_latent_var),
            nn.Tanh(),
            nn.Linear(n_latent_var, n_latent_var),
            nn.Tanh(),
            nn.Linear(n_latent_var, 1)
        )
        
    
    def forward(self):
        raise NotImplementedError
        
    
    def act(self, state, memory):
        state = torch.from_numpy(state).float().to(device)
        action_probs = self.action_layer(state)
        dist = Categorical(action_probs)
        action = dist.sample()
        
        memory.states.append(state)
        memory.actions.append(action)
        memory.logprobs.append(dist.log_prob(action))
        
        return action.item()
    
    
    def evaluate(self, state, action):
        action_probs = self.action_layer(state)
        dist = Categorical(action_probs)
        
        action_logprobs = dist.log_prob(action)
        dist_entropy = dist.entropy()
        
        state_value = self.value_layer(state)
        
        return action_logprobs, torch.squeeze(state_value), dist_entropy
    

class PPO:
    
    def __init__(self, state_dim, action_dim, n_latent_var, lr, betas, gamma,
                K_epochs, eps_clip):
        self.lr = lr
        self.betas = betas
        self.gamma = gamma
        self.eps_clip = eps_clip
        self.K_epochs = K_epochs
        
        self.policy = ActorCritic(state_dim, action_dim, n_latent_var).to(device)
        self.optimizer = torch.optim.Adam(self.policy.parameters(), lr=lr, betas=betas)
        self.policy_old = ActorCritic(state_dim, action_dim, n_latent_var).to(device)
        self.policy_old.load_state_dict(self.policy.state_dict())
        
        self.mseLoss = nn.MSELoss()
        
        
    def update(self, memory):
        # monte carlo estimate of state rewards
        rewards = []
        discounted_reward = 0
        for reward, is_terminal in zip(reversed(memory.rewards), reversed(memory.is_terminals)):
            if is_terminal:
                discounted_reward = 0
            discounted_reward = reward + (self.gamma * discounted_reward)
            rewards.insert(0, discounted_reward)
            
        # normalizing the rewards
        rewards = torch.tensor(rewards, dtype=torch.float32).to(device)
        rewards = (rewards - rewards.mean()) / (rewards.std() + 1e-5)
        
        # convert list to tensor
        old_states = torch.stack(memory.states).to(device).detach()
        old_actions = torch.stack(memory.actions).to(device).detach()
        old_logprobs = torch.stack(memory.logprobs).to(device).detach()
        
        # optimize the policy for K epochs
        for _ in range(self.K_epochs):
            # evaluating old actions and values
            logprobs, state_values, dist_entropy = self.policy.evaluate(old_states, old_actions)
            
            # finding ratio (pi_theta / pi_theta_old)
            ratios = torch.exp(logprobs - old_logprobs.detach())
            
            # finding surrogate loss
            advantages = rewards - state_values.detach()
            surr1 = ratios * advantages
            surr2 = torch.clamp(ratios, 1-self.eps_clip, 1+self.eps_clip) * advantages
            loss = -torch.min(surr1, surr2) + 0.5 * self.mseLoss(state_values, rewards) - 0.01 * dist_entropy
            
            # take gradient step
            self.optimizer.zero_grad()
            loss.mean().backward()
            self.optimizer.step()
            
        # copy new weights into old policy
        self.policy_old.load_state_dict(self.policy.state_dict())
        

In [5]:
def main():
    ### hyperparameters ###
    env_name = "LunarLander-v2"
    # creating environment
    env = gym.make(env_name)
    state_dim = env.observation_space.shape[0]
    action_dim = 4
    render = True # changed from False
    solved_reward = 230         # stop training if avg_reward > solved_reward
    log_interval = 20           # print avg reward in the interval
    max_episodes = 50000        # max training episodes
    max_timesteps = 300         # max timesteps in one episode
    n_latent_var = 64           # number of variables in hidden layer
    update_timestep = 2000      # update policy every n timesteps
    lr = 0.002
    betas = (0.9, 0.999)
    gamma = 0.99                # discount factor
    K_epochs = 4                # update policy for K epochs
    eps_clip = 0.2              # clip parameter for PPO
    random_seed = None
    ### ###

    if random_seed:
        torch.manual_seed(random_seed)
        env.seed(random_seed)

    memory = Memory()
    ppo = PPO(state_dim, action_dim, n_latent_var, lr, betas, gamma, K_epochs, eps_clip)
    print(lr, betas)

    # logging variables
    running_reward = 0
    avg_length = 0
    timestep = 0

    # training step
    for i_episode in range(1, max_episodes+1):
        state = env.reset()
        for t in range(max_timesteps):
            timestep += 1

            # running policy_old
            action = ppo.policy_old.act(state, memory)
            state, reward, done, _ = env.step(action)

            # saving reward and is_terminal 
            memory.rewards.append(reward)
            memory.is_terminals.append(done)

            # update if it's time
            if timestep % update_timestep == 0:
                ppo.update(memory)
                memory.clear_memory()
                timestep = 0

            running_reward += reward
            if render:
                env.render()
            if done:
                break

        avg_length += t

        # stop training if avg_reward > solved_reward
        if running_reward > (log_interval * solved_reward):
            print("######### Solved! #########")
            torch.save(ppo.policy.state_dict(), "./PPO_{}.pth".format(env_name))
            break

        # logging
        if i_episode % log_interval == 0:
            avg_length = int(avg_length/log_interval)
            running_reward = int((running_reward/log_interval))

            print("Episode {} \t Avg Length: {} \t Reward: {}".format(i_episode, avg_length, running_reward))
            running_reward = 0
            avg_length = 0

In [6]:
main()

0.002 (0.9, 0.999)
Episode 20 	 Avg Length: 85 	 Reward: -176
Episode 40 	 Avg Length: 82 	 Reward: -181
Episode 60 	 Avg Length: 83 	 Reward: -164
Episode 80 	 Avg Length: 80 	 Reward: -154
Episode 100 	 Avg Length: 85 	 Reward: -139
Episode 120 	 Avg Length: 82 	 Reward: -137
Episode 140 	 Avg Length: 74 	 Reward: -136
Episode 160 	 Avg Length: 85 	 Reward: -145
Episode 180 	 Avg Length: 83 	 Reward: -118
Episode 200 	 Avg Length: 87 	 Reward: -131
Episode 220 	 Avg Length: 94 	 Reward: -139
Episode 240 	 Avg Length: 95 	 Reward: -130
Episode 260 	 Avg Length: 86 	 Reward: -126
Episode 280 	 Avg Length: 90 	 Reward: -128
Episode 300 	 Avg Length: 96 	 Reward: -127
Episode 320 	 Avg Length: 105 	 Reward: -86
Episode 340 	 Avg Length: 108 	 Reward: -76
Episode 360 	 Avg Length: 110 	 Reward: -100
Episode 380 	 Avg Length: 112 	 Reward: -103
Episode 400 	 Avg Length: 100 	 Reward: -84
Episode 420 	 Avg Length: 118 	 Reward: -68
Episode 440 	 Avg Length: 116 	 Reward: -66
Episode 460 	 A

KeyboardInterrupt: 

In [ ]:
def test():
    ### hyperparameters ###
    env_name = "LunarLander-v2"
    env = gym.make(env_name)
    state_dim = env.observation_space.shape[0]
    action_dim = 4
    render = False
    max_timesteps = 500
    n_latent_var = 64   # hidden layer size
    lr = 0.0007
    betas = (0.9, 0.999)
    gamma = 0.99   # discount factor
    K_epochs = 4   # update policy for K epochs
    eps_clip = 0.2   # clip parameter for PPO
    ###   ###
    
    n_episodes = 3
    max_timesteps = 300
    render = True
    save_gif = False
    
    filename = "PPO_{}.pth".format(env_name)
    directory = "./preTrained/"
    
    memory = Memory()
    ppo = PPO(state_dim, action_dim, n_latent_var, lr, betas, gamma,
             K_epochs, eps_clip)
    ppo.policy_old.load_state_dict(torch.load(os.path.join(directory, filename)))
    
    for ep in range(1, n_episodes+1):
        ep_reward = 0
        state = env.reset()
        for t in range(max_timesteps):
            action = ppo.policy_old.act(state, memory)
            state, reward, done, _ = env.step(action)
            ep_reward += reward
            if render:
                env.render()
            if save_gif:
                img = env.render(mode="rgb_array")
                img = Image.fromarray(img)
                img.save("./gif/{}.jpg".format(t))
            if done:
                break
        
        print("Episode: {}\tReward: {}".format(ep, int(ep_reward)))
        ep_reward = 0
        env.close()